# Min-Max, a perfect Tic Tac Toe player 
In this Notebook, we will use the Min-Max algorithm to create a computer player which will be able to play Tic Tac Toe perfectly. That is, the player will always play the best possible move in a given situation. This player will give us a good benchmark to compare the other players against.

Let's start by importing a few of the utility functions and classes we defined last time and make sure it all still works:

In [1]:
from IPython.display import HTML, display
from tic_tac_toe.Board import Board, GameResult, CROSS, NAUGHT, EMPTY
from util import print_board, play_game
from tic_tac_toe.RandomPlayer import RandomPlayer

board = Board()
player1 = RandomPlayer()
player2 = RandomPlayer()

result = play_game(board, player1, player2)
print_board(board)

if result == GameResult.CROSS_WIN:
    print("Cross won")
elif result == GameResult.NAUGHT_WIN:
    print("Naught won")
else:
    print("Draw")

o,x,x
o,x,
,x,o


Cross won


## The Min-Max algorithm
So, what is this Min-Max algorithm that we want to implement?

The long answer can be found [here](https://en.wikipedia.org/wiki/Minimax). We won't go into that much detail here and just look at the general idea:

Given a board state, we find the best move by simulating all possible continuations from this position and chose the one that is best for us. The one best for us is the one with the best outcome if we assume:

* we always make the move that is best for us (*Maximizes* the game value for us) and 
* our opponent always makes the move that is best for them (and thus worst for us - *Minimizing* the game value for us). 

You can see where the algorithm gets its name from. This algorithm, and variations of it, is very popular for writing game engines that play games like Tik Tak Toe, Chess, Checkers, etc. 

<div class="alert alert-block alert-info">
Side Note: For games with a large number of different board positions like Chess, the algorithm will generally not be able to completely simulate all continuations. In those cases the algorithm will only look ahead a certain number of moves and estimate the value of the position then. Also, more advanced versions of this algorithm, e.g. [alpha / beta pruning](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) are used in those cases.
</div>

Let's look at an exmaple. Given the followin board state and NAUGHT to move next:

In [2]:
example = Board([CROSS  , EMPTY  , CROSS,
                 NAUGHT , NAUGHT , CROSS,
                 EMPTY  , EMPTY  , NAUGHT])
print_board(example)

x,,x
o,o,x
,,o


The following continuations are possible:
![title](./Images/TicTacToe-MinMax-Example1.png)

That is: first NAUGHT, the maximizing player, gets to move, then CROSS, the minimizing player, gets to move and in those cases where the game has not ended at that point, NAUGHT, the maximizing player, gets one more move:
![title](./Images/TicTacToe-MinMax-Example2.png)

We label all final game states according to their value from the point of view of Naught: 
* 1 for a win
* -1 for a loss
* 0 for a draw

![title](./Images/TicTacToe-MinMax-Example3.png)

Now we can back-propagate the scores from the bottom layer to the layer above. According to the algorithm, as it is the maximizing player's turn, we chose the move with the highest score. 

Note that in this initial case, as there is only one possible move and the move thus is forced, we just propagate that value one layer up without having to chose a maximizing move:

![title](./Images/TicTacToe-MinMax-Example4.png)

Now we propagate up again. This time it is the minimizing player's turn, so we propagate the smaller values for each possible move up:

![title](./Images/TicTacToe-MinMax-Example5.png)

Finally, we propagate one more layer up. This time it's the maximizing player again, so we chose the highest possible value of all moves for the position:

![title](./Images/TicTacToe-MinMax-Example6.png)

Now, we know everything we need to know to make a move: 

* The best we can hope for, if both we and our opponent always plays their best move, is a draw (since the value of the current board position is 0). 

* We also know that there is only 1 move in the current situation that will achieve this best case for us: Putting a NAUGHT in the middle spot on the top row.

Note that there are other potential continuation that would also lead to a draw, and some that might even lead to NAUGHT winning. Unfortunately, however, we also know now that if CROSS always plays their best move, we won't ever have a chance to to reach any of these outcomes.

## The Min-Max Player Code
Our code contains 2 player classes which implement the Min Max algorithm for playing Tic Tac Toe:

* [MinMaxAgent.py](https://github.com/fcarsten/tic-tac-toe/blob/master/tic_tac_toe/MinMaxAgent.py): Plays Tic Tac Toe using the Min Max Algorithm in a deterministic way. I.e. if there is more than 1 move with euqal best scores in a given position, this player will always chose the same move.

* [RndMinMaxAgent.py](https://github.com/fcarsten/tic-tac-toe/blob/master/tic_tac_toe/RndMinMaxAgent.py): Plays Tic Tac Toe using the Min Max Algorithm in a non-deterministic way. I.e. if there is more than 1 move with euqal best scores in a given position this player will each time randomly chose between them.

In order to make things a bit more efficient, the players will also store the value for a given board position in an internal cache. This means, that they only have to compute the possible continuations from each position once. It even makes this first computation more efficient, as often different move combination will produce the same board position, which, with the cached result, we don't have to evaluate again.

<div class="alert alert-block alert-info">
Side Note: Even on a moderately fast computer this works quite well due to the small number of possible board positions in Tic Tac Toe: <br/><br/>

While a game can have something like $9! = 362,800$ different possible move combinations, i.e. 9 choices for the first move, 8 choices for the second move, 7 choices for the 3rd move etc down to 1 choice for the last move (and for simpicity ignoring the fact that the game can be over before all squares are occupied - thus reducing the actual number of move combinations), the game can only have $3^9 = 19,683$ different states as each square can only either be empty, have a NAUGHT, or have a CROSS in it (again for simplicity ignoring the fact that these include game states that are impossible in a real game; also ignoring the fact that we could reduce the number of states further by treating symmetric position as being the same). 
</div>

Let's see how they perform.

First we define a small utility function which we call `battle` to repeatedly pit 2 players against each other for a number of games. After the "battle" has finished, the function will return statistics about who won how often.

In [3]:
from tic_tac_toe.Player import Player

def battle(player1: Player, player2: Player, num_games: int = 100000):
    board = Board()
    draw_count = 0
    cross_count = 0
    naught_count = 0
    for _ in range(num_games):
        result = play_game(board, player1, player2)
        if result == GameResult.CROSS_WIN:
            cross_count += 1
        elif result == GameResult.NAUGHT_WIN:
            naught_count += 1
        else:
            draw_count += 1

    print("After {} game we have draws: {}, Player 1 wins: {}, and Player 2 wins: {}.".format(num_games, draw_count,
                                                                                         cross_count, naught_count))

    print("Which gives percentages of draws: {:.2%}, Player 1 wins: {:.2%}, and Player 2 wins:  {:.2%}".format(
        draw_count / num_games, cross_count / num_games, naught_count / num_games))

Let's use this function and to start of with we pit the MinMaxAgent against the RandomPlayer:

In [5]:
from tic_tac_toe.MinMaxAgent import MinMaxAgent

battle(MinMaxAgent(), RandomPlayer())

After 100000 game we have draws: 489, Player 1 wins: 99511, and Player 2 wins: 0.
Which gives percentages of draws: 0.49%, Player 1 wins: 99.51%, and Player 2 wins:  0.00%


And now, with the Random player going first:

In [6]:
battle(RandomPlayer(), MinMaxAgent())

After 100000 game we have draws: 19323, Player 1 wins: 0, and Player 2 wins: 80677.
Which gives percentages of draws: 19.32%, Player 1 wins: 0.00%, and Player 2 wins:  80.68%


And just for validation, the 2 MinMax players against each other:

In [7]:
from tic_tac_toe.RndMinMaxAgent import RndMinMaxAgent

battle(MinMaxAgent(), RndMinMaxAgent())

After 100000 game we have draws: 100000, Player 1 wins: 0, and Player 2 wins: 0.
Which gives percentages of draws: 100.00%, Player 1 wins: 0.00%, and Player 2 wins:  0.00%


As expected and to my relief, we get 100% draws.

# Baseline Part 2

This gives us an additional baseline:

| Player | P1 Win | P2 Win | Draw |
| --- | ---| --- | --- |
| Min Max - Random | 99.5%     | 0%     | 0.5% |
| Random - Min Max | 0%    | 80% | 20% |
| Min Max - Min Max | 0%    | 0% | 100% |

This means, in order to be considered as playing better than pure random, a player should achieve significantly more than 20% draws against a Min-Max player when going first and significantly more than 1% draws when going second.

In the next part we will look at our first player using reinforcement learning: The Tabular Q-Learner.